# Gathering player Wikipedia data

## Part 1: Player Wikipedia pages

In [2]:
import csv
import requests
import time

playerinfo = []
with open("USWNT/playerdata.csv", "r", newline = "") as infile:
    csvin = csv.reader(infile)
    headers = next(csvin)
    for line in csvin:
        playerinfo.append(line)

firstname = headers.index("Firstname")
surname = headers.index("Surname")

wikiplayerurls = []
for singleplayer in playerinfo:
    playerfirstname = singleplayer[firstname]
    playersurname = singleplayer[surname]
    if playersurname == "Campbell" or playersurname == "Smith" or playersurname == "Williams":
        wikiplayerurl = "https://en.wikipedia.org/wiki/" + playerfirstname + "_" + playersurname + "_(soccer)"
    else:
        wikiplayerurl = "https://en.wikipedia.org/wiki/" + playerfirstname + "_" + playersurname
    wikiplayerurls.append(wikiplayerurl)

uniquewikiurls = set(wikiplayerurls)

outfile = open('Playerwiki/wikifilenames.txt', 'w')
for url in uniquewikiurls:
    last_slash = url.rindex("/")
    filename = url[last_slash + 1:] + ".html"
    print(filename, file=outfile)

outfile.close()

for fn in wikifilenames:
    outfile = open(fn, "w")
    result = requests.get(url)
    print(result.text, file=outfile)
    outfile.close()
    time.sleep(2)

    

## Part 2: Finding player college teams and the Wikipedia page for the teams

In [20]:
from bs4 import BeautifulSoup
from lxml import html

infile = open('Playerwiki/wikifilenames.txt', 'r')
wikifilenames = infile.read().split()

playercollegelinks = []

for playerwiki in wikifilenames:
    playercollegelink = []
    infile = open(playerwiki, "rb")
    soup = BeautifulSoup(infile, 'html.parser')
    soupstring = str(soup)       
    try:
        findcollege = soup.find('th', text="College career")
        findcollegestring = str(findcollege)
        secondhalf = soupstring[soupstring.index(findcollegestring):]
        findsenior = secondhalf.find('Senior')
        whatineed = secondhalf[:findsenior]
        tree=html.fromstring(whatineed)
        collegelink = tree.xpath('//a/@href')
        playercollegelink.append(playerwiki)
        for link in collegelink:
            playercollegelink.append(link)
    except:
        playercollegelink.append(playerwiki)
        playercollegelink.append("NoCollege")
        
    playercollegelinks.append(playercollegelink)


import csv

headers = ['Filename', 'College Team URL']

outfile = open('Collegeteam/collegeteamlinks.csv', 'w')
csv_out = csv.writer(outfile)
csv_out.writerow(headers)
csv_out.writerows(playercollegelinks)
outfile.close()    

        
    

In [1]:
import csv

collegelinks = []
with open('Collegeteam/collegeteamlinks.csv', 'r', newline = "") as infile:
    csvin = csv.reader(infile)
    headers = next(csvin)
    for line in csvin:       
        collegelinks.append(line)

finalcollegelinks = []        
for links in collegelinks:
    actuallinks = links[1:]
    for actuallink in actuallinks:
        if actuallink.startswith("#")==True:
            pass
        elif actuallink=="NoCollege":
            pass
        else:
            finalcollegelinks.append(actuallink[actuallink.rindex('/'):]) # Only grabs the end of the link for the filename, removing the /wiki portion

uniquecollegelinks = set(finalcollegelinks)            
outfile = open("Collegeteam/collegeteamurls.txt", "w")
for link in uniquecollegelinks:
     print(link, file=outfile)
outfile.close()



NoCollege
NoCollege


In [8]:
import requests
import time

infile = open("Collegeteam/collegeteamurls.txt", "r")

collegelinks = infile.read().split()

for link in collegelinks:
    linkbuild = "https://en.wikipedia.org/wiki" + link
        
    outfile = open('Collegeteam' + link + '.html', 'w')
    result = requests.get(linkbuild)
    print(result.text, file=outfile)
    time.sleep(2)
    outfile.close()

## Part 3: Finding the Wiki page for the actual university

In [39]:
from lxml import html

infile = open("Collegeteam/collegeteamurls.txt", "r")

collegelinks = infile.read().split()

filenames = []
for link in collegelinks:
    filename = 'Collegeteam' + link + '.html'
    filenames.append(filename)

collegenames = []    
for teamfile in filenames:
    infile = open(teamfile, 'rb')
    page = infile.read()
    tree=html.fromstring(page)    
    college = tree.xpath('//tr[contains(th,"University")]/td/a/@href')
    
    collegename = []
    collegename.append(teamfile)
    for link in college:
        collegelink = 'Collegewiki' + link[link.rindex('/'):] # Removes the /wiki from the beginning of the url
        collegename.append(collegelink)
    collegenames.append(collegename)


import csv

headers = ['College Team Filename', 'College Name']
outfile = open('Collegewiki/collegelinks.csv', 'w')
csv_out = csv.writer(outfile)
csv_out.writerow(headers)
csv_out.writerows(collegenames)
outfile.close()    


In [ ]:
import csv
import requests
import time

collegelinks = []
with open("Collegewiki/collegelinks.csv", "r", newline = "") as infile:
    csvin = csv.reader(infile)
    headers = next(csvin)
    for line in csvin:
        collegelinks.append(line)

collegename = headers.index("College Name")

for link in collegelinks:
    collegefilename = link[collegename]
    
    collegeurl = 'https://wikipedia.org/wiki' + collegefilename[collegefilename.rindex('/'):]
        
    outfile = open(collegefilename + '.html', 'w')
    result = requests.get(collegeurl)
    print(result.text, file=outfile)
    time.sleep(2)
    outfile.close()

In [ ]:
import csv
from lxml import html

collegelinks = []
with open("Collegewiki/collegelinks.csv", "r", newline = "") as infile:
    csvin = csv.reader(infile)
    headers = next(csvin)
    for line in csvin:
        collegelinks.append(line)

collegename = headers.index("College Name")

allcollegedata = []

for link in collegelinks:
    collegefilename = link[collegename] + '.html'
    
    infile = open(collegefilename, 'rb')
    page = infile.read()
    tree=html.fromstring(page)    
    
    schoolname = tree.xpath('//h1/text()')
    schooltype = tree.xpath('//tr[contains(th,"Type")]/td/a/text()')
    students = tree.xpath('//tr[contains(th,"Students")]/td/text()')
    location = tree.xpath('//tr[contains(th,"Location")]/td/div/a/text()')
    
    collegedata = []
    collegedata.append(collegefilename)
    collegedata.append(",".join(schoolname))
    collegedata.append(",".join(location))
    collegedata.append(",".join(schooltype))
    collegedata.append(",".join(students))
    allcollegedata.append(collegedata)

headers = ['File Name', 'School Name', 'Location', 'Type', 'Enrollment']

outfile = open('Collegewiki/collegedata.csv', 'w')
csv_out = csv.writer(outfile)
csv_out.writerow(headers)
csv_out.writerows(allcollegedata)
outfile.close()    